<a href="https://colab.research.google.com/github/erica-mccormick/widespread-bedrock-water-use/blob/main/Code_Part1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **PART 1: Code for *Evidence for widespread woody plant use of water stored in bedrock***
**Erica McCormick, David Dralle, W. Jesse Hahm, Alison Tune, Logan Schmidt, Dana Chadwick, and Daniella Rempe**

---

All data products are available in the [Hydroshare repository](https://doi.org/10.4211/hs.a2f0d5fd10f14cd189a3465f72cba6f3) and as GEE assets (Images and ImageCollections, see Part 2).

See [Github](https://github.com/erica-mccormick/widespread-bedrock-water-use) or [Zenodo]() for Part 2 and more information on data inputs and products.

For more information, see [website](https://erica-mccormick.github.io/widespread-bedrock-water-use/).

---

#**Part 1: Calculate $D_{bedrock}$ and $S_{bedrock}$ for CONUS**


Press shift+enter on each chunk while highlighted to run code. The output rasters from this script are $S_r$ and $D_{max}$ for CONUS. $S_{bedrock}$ and $D_{bedrock, 2004-2017}$ can be calculated by subtracting $S_{soil}$ (see Methods). The products from this script can be found on [hydroshare](https://www.hydroshare.org/resource/e7ad140edaf54d69ba4f1cf1ec8e7f73/). The protocol for $S_r$ follows that established by [Wang-Erlandsson et al. (2016)](https://doi.org/10.5194/hess-20-1459-2016) and [Dralle et al. (2020)](https://10.5194/hess-25-2861-2021).

### Authenticate Google Earth Engine (GEE) account. First time users sign-up [here](https://earthengine.google.com/new_signup/).

In [ ]:
import ee
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=oSXBjESbXcvL1R1R9m0G-yfQaWugDfOEKXqVHtYd_Z4&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AY0e-g4_hoiNN47YIcNmC7-Aie5O70XrOXJOr-x-Jrql-xN0spNWDzuGQCs

Successfully saved authorization token.


### GEE assets used in this script:

Publically Available Assets:

In [ ]:
prism = ee.ImageCollection("OREGONSTATE/PRISM/AN81d")
pml = ee.ImageCollection("CAS/IGSNRR/PML/V2")
USGS_landcover = ee.ImageCollection("USGS/NLCD")
modis_landcover = ee.ImageCollection("MODIS/006/MCD12Q1")
snow_cover = ee.ImageCollection("MODIS/006/MOD10A1").select('NDSI_Snow_Cover')
bess = ee.Image('users/daviddralle/bessv2')

Personal Assets:

In [ ]:
# Regions
ca = ee.Feature(ee.FeatureCollection("users/daviddralle/ca_et/CA").first())
conus = ee.FeatureCollection('users/ericamccormick/20_RockMoisture/geometries/conus_20m')
texas = ee.FeatureCollection('users/ericamccormick/20_RockMoisture/geometries/TX')

In [ ]:
# gNATSGO derived (Soil Survey Staff, 2019):
us_soil = ee.Image('users/ericamccormick/20_RockMoisture/products/gNATSGO/Ssoil_500m')
densic = ee.Image('users/ericamccormick/20_RockMoisture/products/gNATSGO/densic_weavg_lower_reprojected').reproject(crs='EPSG:4326',scale=90) # Soil Survey Staff, 2020
paralithic = ee.Image('users/ericamccormick/20_RockMoisture/products/gNATSGO/paralithic_weavg_lower_reprojected').reproject(crs='EPSG:4326',scale=90) # Soil Survey Staff, 2020
lithic = ee.Image("users/ericaelmstead/20_RockMoisture/gNATSGO/lithic_weavg_lower_reprojected").reproject(crs='EPSG:4326',scale=90) # Soil Survey Staff, 2020

### Load packages

In [ ]:
import datetime
from IPython.display import Image
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
plt.rcParams['pdf.fonttype'] = 42
from google.colab import files
from IPython.display import Image
%matplotlib inline

For running analyses in GoogleColab, download datasets and .tiffs to GoogleDrive and import directly from there. To mount your Drive, find the file name for the drive folder you wish to connect to:

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#**Create woody vegetation and shallow bedrock masks**

In [ ]:
# Get the crs of the layer you want to match (here using prism as a test)
prism_image = ee.ImageCollection("OREGONSTATE/PRISM/AN81d").first().reproject(crs='EPSG:4326', scale = 500)

prism_projection = prism_image.projection();
prism_projection.getInfo()

{'crs': 'EPSG:4326',
 'transform': [0.004491576420597608, 0, 0, 0, -0.004491576420597608, 0],
 'type': 'Projection'}

## Get binary soil depth > 1.5 and all soil depth pixels = 1 layers

In [ ]:
# Get all of the bedrock pixels = 1 for counting
all_bedrock_pixels = paralithic.add(densic).add(lithic)
all_bedrock_pixels = all_bedrock_pixels.gt(-1)

In [ ]:
# mask places with greater than 1.5m soil
depth = 150

m = paralithic.select('b1').lte(depth)
m1 = densic.select('b1').lte(depth)
m2 = lithic.select('b1').lte(depth)

bedrock_depth_include = m.add(m1).add(m2) #could add.gt(0) here to make sure all values are one, but we are counting so its not a big deal plus theres very little to no overlap between bedrock types

included_depth = all_bedrock_pixels.updateMask(bedrock_depth_include)	

## Coarsen resolution using sum to 500m

In [ ]:

count_all = all_bedrock_pixels.reduceResolution(**{
  'reducer': ee.Reducer.count()
  }).reproject(crs=prism_projection)


count_under_threshold = included_depth.reduceResolution(**{
  'reducer': ee.Reducer.count()
  }).reproject(crs=prism_projection)

In [ ]:
fraction = count_under_threshold.divide(count_all).multiply(100)
keep_shallowbedrock = fraction.gte(100)

## Export

In [ ]:
task_config = {
        'region': conus.geometry(),
        'fileFormat': 'GeoTIFF',
        'fileNamePrefix': 'mask_shallowBedrock',
        'image': keep_shallowbedrock.toDouble().clip(conus.geometry()),
        'description': 'mask_shallowBedrock',
        'scale': 500,
        'maxPixels': 10000000000000
    }

task=ee.batch.Export.image.toDrive(**task_config)
task.start()

In [ ]:
task_config = {
        'region': conus.geometry(),
        'image': keep_shallowbedrock,
        'description': 'shallowBedrock',
        'assetId' : 'users/ericamccormick/mask_shallowBedrock',
        'scale': 500,
        'maxPixels': 10000000000000
    }

task=ee.batch.Export.image.toAsset(**task_config)
task.start()

## Repeat for NLCD 30m forest and shrubland classifications

In [ ]:
usgs_nlcd = ee.ImageCollection("USGS/NLCD_RELEASES/2016_REL")

In [ ]:
landsort = usgs_nlcd.sort('system:time_start', False) #sort backcwards to get most recent image
landsort = ee.Image(landsort.first()) # make sure to get just first image
img1 = landsort.select('landcover').reproject(crs='EPSG:4326',scale=30)

## Get shrub at bedrock.lt(depth): 
shrub = img1.remap([52],[1],0) #eq(52) #mask to get all values = 52
#shrub_final = shrub.multiply(ee.Image(2)) # set shrubs as value 2, everything else 0

# get classes so forested is 0s with 1s where values 41,42,43. Use function 'reclassify'
forested = img1.remap([41,42,43],[1,1,1],0) #forested.remap([from],[to],defaultVal) , defaultval sets everything else to zerofore

## add forested_final and shrub_final and return a mask with 1s and 0s:
woody_veg = ee.Image(0).add(forested).add(shrub) #.gt(0) #final binary tiff where 1s are places with woody veg

In [ ]:
woody_pixels_only = img1.updateMask(woody_veg)

In [ ]:
count_landcover_all = img1.reduceResolution(**{
  'reducer': ee.Reducer.count(),
  'maxPixels': 500
  }).reproject(crs=prism_projection)


count_woody = woody_pixels_only.reduceResolution(**{
  'reducer': ee.Reducer.count(),
  'maxPixels':500
  }).reproject(crs=prism_projection)

In [ ]:
## do division and thresholding here
fraction_woody = count_woody.divide(count_landcover_all).multiply(100)
keep_woodyveg = fraction_woody.gte(75)

In [ ]:
task_config = {
        'region': conus.geometry(),
        'fileFormat': 'GeoTIFF',
        'fileNamePrefix': 'mask_woodyVeg',
        'image': keep_woodyveg.toDouble().clip(conus.geometry()),
        'description': 'mask_woodyVeg',
        'scale': 500,
        'maxPixels': 10000000000000
    }

task=ee.batch.Export.image.toDrive(**task_config)
task.start()

In [ ]:
task_config = {
        'region': conus.geometry(),
        'image': keep_woodyveg,
        'description': 'mask_woodyVeg',
        'assetId' : 'users/ericamccormick/mask_woodyVeg',
        'scale': 500,
        'maxPixels': 10000000000000
    }

task=ee.batch.Export.image.toAsset(**task_config)
task.start()

#**Calculate $S_{r}$ for CONUS**

### Assuming $F_{out}=ET$, calculate $F_{out}$ time series without snow correction (following [Wang-Erlandsson et al (2016)](https://hess.copernicus.org/articles/20/1459/2016/)) and with a snow correction.

In [ ]:
#Flux is stored as mm/day, and duration between samples is 8 days
multiplier = 8.0 

#### USING SNOW CORRECTION METHOD ####
# threshold of non-snow-fraction above which we assume that there is 
# negligible snow in the image
threshold = 0.9
# mapping function for creating ET time series but set to zero (Fout = 0) during
# periods when snow is present
def sumpml_snow(image):
  #First, get percent snow coverage for nearest datetime
  current_datetime = datetime.datetime.now().strftime('%Y-%m-%d')
  nearest_snow_image = snow_cover.filterDate(image.get('system:time_start'), 
                                             current_datetime).first() 
  #reproject
  nearest_snow_image= nearest_snow_image.reproject(crs='EPSG:4326',scale=500)
  non_snow_frac = ee.Image(1).subtract(nearest_snow_image.divide(100))
  #assign non-snow fraction to 1 if nodata
  non_snow_frac=non_snow_frac.unmask(1)

  # zero multiplier if there is less than threshold non snow frac
  # i.e. if there is significant snow in the image
  snow_multiplier = non_snow_frac.gt(threshold)

  # get total surface evap and transpiration, zero out if significant snow
  temp = image.select('Es').add(image.select('Ec')).multiply(snow_multiplier)

  #get the first band of this new temporary image, rename it to 'ET', 
  #reproject it, and then multiply by 8
  temp = temp.select([0], ['ET']).reproject(
      crs='EPSG:4326',scale=500
      ).multiply(multiplier)
  #temp should now be total ET in mm over the time window between images
  #assign the datetime stamp and the index from the original image
  temp = temp.set('system:time_start', image.get('system:time_start'))
  temp = temp.set('system:index', image.get('system:index'))
  return temp

#now, actually map the pml image collection w/ this function,
# in order to make a new combined ET image collection
et_snow_corr = ee.ImageCollection(pml.map(sumpml_snow).select('ET'))

In [ ]:
# function use to calculate A_{t_n} values and running storage deficit
def get_AD(et, yearstart, yearend):
  et_current = et.filterDate(yearstart, yearend)
  prism = ee.ImageCollection('OREGONSTATE/PRISM/AN81d').select('ppt')

  # Create starting image for iterator used to calculate A_{t_n}
  starter = ee.Image(et_current.first())
  # set the time of the starter image to 8 days before first ET image
  time0 = starter.get('system:time_start').getInfo() - 691000000
  first = ee.List([
    ee.Image(0).set('system:time_start', time0).select([0], ['A']).toDouble()
  ])

  # for each item in the ET (F_out) time series, 
  # get cumulative precip (F_in) since previous F_out
  # Take the difference between F_out and F_in; this is A
  def getA(image, thelist):
    previous = ee.Image(ee.List(thelist).get(-1))
    startdate = previous.get('system:time_start')
    enddate = image.get('system:time_start')
    windowed_prism = prism.filterDate(startdate,enddate)
    prism_total = windowed_prism.reduce(ee.Reducer.sum())
    Atemp = image.subtract(prism_total)
    Atemp = Atemp.select([0], ['A']).toDouble()
    Atemp = Atemp.set({'system:time_start':image.get('system:time_start')})
    return ee.List(thelist).add(Atemp)
  # iterate using getA() 
  A = ee.ImageCollection(ee.List(et_current.iterate(getA, first)))

  # add band to keep track of running storage deficit calculation
  def add_deficit_band(image):
    return image.addBands(ee.Image.constant(0).rename('D')).toDouble()
  AD_initial = A.map(add_deficit_band)
  starter = ee.Image(AD_initial.first()).multiply(0)

  ts = AD_initial.first().get('system:time_start')
  starter = starter.set('system:time_start', ts)

  first = ee.List([starter])
  # iterator we will use to calculate running storage deficit
  def accumulate(image, thelist):
    # get image with previous D values
    previous = ee.Image(ee.List(thelist).get(-1))
    previousD = previous.select('D')

    # get current value of A = F_out - F_in
    currentA = image.select('A')

    # D is just previous D plus current A
    newD = previousD.add(currentA)

    # if storage deficit is < 0, set it to zero (can only be positive)
    newD = newD.multiply(newD.gt(0))

    # add to list over which we are iterating
    tempAD = currentA.addBands(newD)
    return ee.List(thelist).add(tempAD)

  # Get storage deficit time series, take maximum observed storage deficit
  AD = ee.ImageCollection(ee.List(AD_initial.iterate(accumulate, first)))
  return AD

### Use `get_AD()` function to calculate max storage deficits

In [ ]:
# get max storage deficit using non-snow-corrected and snow-corrected F_out
yearstart = '2003'
yearend = '2017'

AD_snow_corr = get_AD(et_snow_corr, yearstart, yearend)

# Equivelant to Sr
maxD_snow_corr = AD_snow_corr.select('D').max().select([0],['S_R_snow_corr'])


### Apply woody veg and shallow bedrock masks and add in additional ET>P mask

In [ ]:
# ET>P
etsum = et_snow_corr.filterDate('2003', '2017').sum().clip(conus.geometry()).reproject(crs='EPSG:4326',scale=500)
prism = ee.ImageCollection('OREGONSTATE/PRISM/AN81m').select('ppt')
pptsum = prism.filterDate('2003', '2017').sum().clip(conus.geometry()).reproject(crs='EPSG:4326',scale=500)
ppt_minus_et = pptsum.subtract(etsum)
mask_ET_gt_PPT = ppt_minus_et.gte(0)

# Multiply masks together
mask = mask_ET_gt_PPT.multiply(keep_woodyveg).multiply(keep_shallowbedrock)

mask_reproj = mask.reproject(
      crs='EPSG:4326',scale=500)

# create multiband raster with S_R values and snowpack band
# mask by land cover type
#to_save = maxD.addBands(maxD_snow_corr).addBands(mean_winter_snow_cover).addBands(abs_diff)

# add difference between snowb
#Sr_tosave = maxD_snow_corr.updateMask(mask_reproj)
Sr = maxD_snow_corr.updateMask(mask_reproj).clip(conus.geometry())


In [ ]:
task_config = {
        'region': conus.geometry(),
        'fileFormat': 'GeoTIFF',
        'fileNamePrefix': 'Sr',
        'image': Sr.toDouble(),
        'description': 'pptsum2',
        'scale': 500,
        'maxPixels': 10000000000000
    }

task=ee.batch.Export.image.toDrive(**task_config)
task.start()

## Export

Only run this cell if you intend to export the dataset from Earth Engine to your Google Drive. This operation will take several hours to complete. You can check the status of the export on [https://code.earthengine.google.com/](https://code.earthengine.google.com/).

In [ ]:
task_config = {
        'region': conus.geometry(),
        'fileFormat': 'GeoTIFF',
        'fileNamePrefix': 'Sr_' + str(yearstart) + '_' + str(yearend) + '_Mask',
        'image': Sr.toDouble(),
        'description': 'Sr',
        'scale': 500,
        'maxPixels': 10000000000000
    }

task=ee.batch.Export.image.toDrive(**task_config)
task.start()

In [ ]:
task_config = {
        'region': conus.geometry(),
        'image': Sr,
        'description': 'Sr_mask',
        'assetId' : 'users/ericamccormick/Sr' + str(yearstart) + '_' + str(yearend) + '_Mask',
        'scale': 500,
        'maxPixels': 10000000000000
    }

task=ee.batch.Export.image.toAsset(**task_config)
task.start()

#**Calculate $D_{max, 2003-2017}$ for CONUS**

In [ ]:
yearstart = '2003'
yearend = '2017'

for year in range(2003,2017):
  yearstart = str(year) + '-10-01'
  yearend = str(year+1) + '-10-01'

  AD_snow_corr = get_AD(et_snow_corr, yearstart, yearend)

  def add_time_band(image):
    t = ee.Number(image.get('system:time_start')).toLong() #changed from .toLong()
    timeimage = ee.Image(t).select([0],['t'])
    image = image.addBands(timeimage)
    image = image.toLong()
    return image

  AD_snow_corr = AD_snow_corr.map(add_time_band, opt_dropNulls=True)

  def greater_than_zero(image):
    temp = image.select('D').gt(0).select([0],['D_positive']).toLong() 
    image = image.addBands(temp)
    return image

  D_positive = AD_snow_corr.select(['t','D']).map(greater_than_zero)
  # D_positive has three bands: t, D, and D_positive

  # // create a starting image for iterator
  starter = ee.Image(D_positive.first())
  starter = starter.addBands(ee.Image(0).select([0],['cumulative_time']))
  first = ee.List([starter])

  def accumulate(image, thelist):
    previous = ee.Image(ee.List(thelist).get(-1))
    # // accumulate and reset counter where image=0
    added = image.select('D_positive').add(previous.select('cumulative_time')).multiply(image.select('D_positive'))
    added = added.select([0],['cumulative_time'])
    image = image.addBands(added)
    # image has 4 bands t, D, D_positive, and cumulative_time
    return ee.List(thelist).add(image)

  cumulative_list = ee.List(D_positive.iterate(accumulate, first))
  cumulative_list = cumulative_list.remove(cumulative_list.get(0))
  # get rid of first item in list, keep everything else, convert to imagecollection
  cumulative = ee.ImageCollection(cumulative_list)
  # argmax = cumulative.qualityMosaic('D').select(['t','D','cumulative_time'])
  argmax = cumulative.qualityMosaic('D').select(['t','cumulative_time','D'])

  to_save = argmax.updateMask(mask_reproj).clip(conus.geometry())
  to_save = to_save.toDouble() # Must comment this out to send to asset.

  task_config = {'region': conus.geometry(),
          'fileFormat': 'GeoTIFF',
          'fileNamePrefix': 'Dmax_' + str(year+1) + '_Mask', 
          'image': to_save,
          'description': 'Dmax_' + str(year+1) + '_Mask',
          'scale': 500,
          'maxPixels': 10000000000000
      }

  task=ee.batch.Export.image.toDrive(**task_config)
  task.start()


Export annual to GEE asset:

In [ ]:
yearstart = '2003'
yearend = '2017'

for year in range(2003,2017):
  yearstart = str(year) + '-10-01'
  yearend = str(year+1) + '-10-01'

  AD_snow_corr = get_AD(et_snow_corr, yearstart, yearend)

  def add_time_band(image):
    t = ee.Number(image.get('system:time_start')).toLong() #changed from .toLong()
    timeimage = ee.Image(t).select([0],['t'])
    image = image.addBands(timeimage)
    image = image.toLong()
    return image

  AD_snow_corr = AD_snow_corr.map(add_time_band, opt_dropNulls=True)

  def greater_than_zero(image):
    temp = image.select('D').gt(0).select([0],['D_positive']).toLong() 
    image = image.addBands(temp)
    return image

  D_positive = AD_snow_corr.select(['t','D']).map(greater_than_zero)
  # D_positive has three bands: t, D, and D_positive

  # // create a starting image for iterator
  starter = ee.Image(D_positive.first())
  starter = starter.addBands(ee.Image(0).select([0],['cumulative_time']))
  first = ee.List([starter])

  def accumulate(image, thelist):
    previous = ee.Image(ee.List(thelist).get(-1))
    # // accumulate and reset counter where image=0
    added = image.select('D_positive').add(previous.select('cumulative_time')).multiply(image.select('D_positive'))
    added = added.select([0],['cumulative_time'])
    image = image.addBands(added)
    # image has 4 bands t, D, D_positive, and cumulative_time
    return ee.List(thelist).add(image)

  cumulative_list = ee.List(D_positive.iterate(accumulate, first))
  cumulative_list = cumulative_list.remove(cumulative_list.get(0))
  # get rid of first item in list, keep everything else, convert to imagecollection
  cumulative = ee.ImageCollection(cumulative_list)
  # argmax = cumulative.qualityMosaic('D').select(['t','D','cumulative_time'])
  argmax = cumulative.qualityMosaic('D').select(['t','cumulative_time','D'])

  to_save = argmax.updateMask(mask_reproj)

  to_save = to_save.clip(conus.geometry()) # added 10/12/20 by Erica from David-Daniella chat

  task_config = {
          'region': conus.geometry(),
          'image': to_save,
          'description': 'US_ANNUALargmax_allrock150_' + str(year+1),
          'assetId' : 'users/ericaelmstead/US_ANNUALargmax_allrock150_' + str(year+1),
          'scale': 1000,
          'maxPixels': 10000000000000
      }

  task=ee.batch.Export.image.toAsset(**task_config)
  task.start()

#**Subtract $S_{soil}$ to calculate $S_{bedrock}$ and $D_{bedrock,Y}$**


###Get $S_{soil}$

In [ ]:
ssoil_mm = ee.Image("users/ericaelmstead/20_RockMoisture/gNATSGO/US_AWS_reprojected").clip(conus.geometry()).multiply(10).reproject(crs = 'EPSG:4326', scale = 90)

# Specify for mean soil to be used when scaling to 500m
mean_soil = ssoil_mm.reduceResolution(**{
  'reducer': ee.Reducer.mean()
})


In [ ]:
task_config = {
        'region': conus.geometry(),
        'fileFormat': 'GeoTIFF',
        'fileNamePrefix': 'Ssoil_500m',
        'image': mean_soil.toDouble(),
        'description': 'Ssoil_500m',
        'scale': 500,
        'maxPixels': 10000000000000
    }

task=ee.batch.Export.image.toDrive(**task_config)
task.start()

In [ ]:
task_config = {
        'region': conus.geometry(),
        'image': mean_soil,
        'description': 'Ssoil_500m',
        'assetId' : 'users/ericamccormick/Ssoil_500m',
        'scale': 500,
        'maxPixels': 10000000000000
    }

task=ee.batch.Export.image.toAsset(**task_config)
task.start()

###Get $S_{bedrock}$

In [ ]:
# Sbedrock = Sr - Ssoil
Sbedrock = Sr.subtract(mean_soil).clip(conus.geometry())

In [ ]:
task_config = {
        'region': conus.geometry(),
        'fileFormat': 'GeoTIFF',
        'fileNamePrefix': 'Sbedrock',
        'image': Sbedrock.toDouble(),
        'description': 'Sbedrock',
        'scale': 500,
        'maxPixels': 10000000000000
    }

task=ee.batch.Export.image.toDrive(**task_config)
task.start()

In [ ]:
task_config = {
        'region': conus.geometry(),
        'image': Sbedrock,
        'description': 'Sbedrock',
        'assetId' : 'users/ericamccormick/Sbedrock',
        'scale': 500,
        'maxPixels': 10000000000000
    }

task=ee.batch.Export.image.toAsset(**task_config)
task.start()

### Get $D_{bedrock,Y}$

In [ ]:
yearstart = '2003'
yearend = '2017'

for year in range(2003,2017):
  yearstart = str(year) + '-10-01'
  yearend = str(year+1) + '-10-01'

  AD_snow_corr = get_AD(et_snow_corr, yearstart, yearend)

  def add_time_band(image):
    t = ee.Number(image.get('system:time_start')).toLong() #changed from .toLong()
    timeimage = ee.Image(t).select([0],['t'])
    image = image.addBands(timeimage)
    image = image.toLong()
    return image

  AD_snow_corr = AD_snow_corr.map(add_time_band, opt_dropNulls=True)

  def greater_than_zero(image):
    temp = image.select('D').gt(0).select([0],['D_positive']).toLong() 
    image = image.addBands(temp)
    return image

  D_positive = AD_snow_corr.select(['t','D']).map(greater_than_zero)
  # D_positive has three bands: t, D, and D_positive

  # // create a starting image for iterator
  starter = ee.Image(D_positive.first())
  starter = starter.addBands(ee.Image(0).select([0],['cumulative_time']))
  first = ee.List([starter])

  def accumulate(image, thelist):
    previous = ee.Image(ee.List(thelist).get(-1))
    # // accumulate and reset counter where image=0
    added = image.select('D_positive').add(previous.select('cumulative_time')).multiply(image.select('D_positive'))
    added = added.select([0],['cumulative_time'])
    image = image.addBands(added)
    # image has 4 bands t, D, D_positive, and cumulative_time
    return ee.List(thelist).add(image)

  cumulative_list = ee.List(D_positive.iterate(accumulate, first))
  cumulative_list = cumulative_list.remove(cumulative_list.get(0))
  # get rid of first item in list, keep everything else, convert to imagecollection
  cumulative = ee.ImageCollection(cumulative_list)
  # argmax = cumulative.qualityMosaic('D').select(['t','D','cumulative_time'])
  argmax = cumulative.qualityMosaic('D').select(['t','cumulative_time','D'])

  dmax = argmax.select('D').updateMask(mask_reproj)
  dbedrock = dmax.subtract(mean_soil).clip(conus.geometry())

  task_config = {'region': conus.geometry(),
          'fileFormat': 'GeoTIFF',
          'fileNamePrefix': 'Dbedrock_' + str(year+1) + '_nomask', 
          'image': dbedrock.toDouble(),
          'description': 'Dbedrock' + str(year+1) + '_nomask',
          'scale': 500,
          'maxPixels': 10000000000000
      }

  task=ee.batch.Export.image.toDrive(**task_config)
  task.start()



#**References**



Daly, C., Halbleib, M., Smith, J. I., Gibson, W. P., Doggett, M. K., Taylor, G. H., Curtis, J., and Pasteris, P. P.: Physiographically Sensitive
15 Mapping of Climatological Temperature and Precipitation across the Conterminous United States, International Journal of Climatology,
28, 2031–2064, https://doi.org/10.1002/joc.1688, 2008.


Dralle, D. N., W. J. Hahm, D. M. Rempe (2020). Dataset for "Accounting for snow in the estimation of root-zone water storage capacity from precipitation and evapotranspiration fluxes", HydroShare, http://www.hydroshare.org/resource/ee45c2f5f13042ca85bcb86bbfc9dd37


Soil Survey Staff: Gridded National Soil Survey Geographic (gNATSGO) Database for the Conterminous United States, Tech. rep., United
States Department of Agriculture, Natural Resources Conservation Service, available online at /https://nrcs.app.box.com/v/soils, 2019.

Wang-Erlandsson, L., Bastiaanssen, W. G. M., Gao, H., Jägermeyr, J., Senay, G. B., van Dijk, A. I. J. M., Guerschman, J. P., Keys, P. W.,
Gordon, L. J., and Savenije, H. H. G.: Global Root Zone Storage Capacity from Satellite-Based Evaporation, Hydrology and Earth System
Sciences, 20, 1459–1481, https://doi.org/10.5194/hess-20-1459-2016, 2016.


Yang, L., J. S. D. P. H. C. G. L. C. A. C. C. D. J. F. J. F. M. G. B. R. M. and Xian, G.: A New Generation of the United States National Land
20 Cover Database: Requirements, Research Priorities, Design, and Implementation Strategies, pp. 108–123, https://developers.google.com/
earth-engine/datasets/catalog/USGS_NLCD#citations, 2018.


Zhang, Y., Kong, D., Gan, R., Chiew, F. H. S., McVicar, T. R., Zhang, Q., and Yang, Y.: Coupled Estimation of 500 m and 8-Day
Resolution Global Evapotranspiration and Gross Primary Production in 2002–2017, Remote Sensing of Environment, 222, 165–182,
305 https://doi.org/10.1016/j.rse.2018.12.031, 2019.